########## choose stock list (aka market) ##################

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [2]:
## To acquire stock list in NASDAQ ###
ex1 = pd.read_csv('stock_info_vn.csv')
df = pd.DataFrame(ex1['Code'].astype('str') + '.VN')
df = df.rename(columns={'Code' : 'ticker'})
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# df = df[:20]

### Acquire informations from yfinance function >> .info

In [3]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [4]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


### Drop NAN #############
df = df.dropna()
#### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_vn.csv',index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/126IBC.VN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=126IBC.VN&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/3TAUDIT.VN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=3TAUDIT.VN&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AA.VN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AA.VN&crumb=yVHB045cuAA
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAAA.VN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDet

############################################################################

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_vn.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Consumer Cyclical' 'Real Estate' 'Financial Services'
 'Consumer Defensive' 'Industrials' 'Utilities' 'Technology' 'Healthcare'
 'Basic Materials' 'Energy']
industry ['Packaging & Containers' 'Real Estate - Diversified' 'Capital Markets'
 'Farm Products' 'Real Estate Services' 'Airports & Air Services'
 'Confectioners' 'Conglomerates' 'Engineering & Construction'
 'Beverages—Brewers' 'Building Products & Equipment' 'Waste Management'
 'Electrical Equipment & Parts' 'Utilities—Regulated Electric'
 'Information Technology Services' 'Auto Parts'
 'Drug Manufacturers—Specialty & Generic' 'Agricultural Inputs'
 'Specialty Chemicals' 'Electronics & Computer Distribution'
 'Paper & Paper Products' 'Real Estate—Development' 'Chemicals'
 'Marine Shipping' 'Real Estate—Diversified' 'Asset Management'
 'Packaged Foods' 'Specialty Retail' 'Oil & Gas Integrated'
 'Utilities—Renewable' 'Infrastructure Operations' 'Steel'
 'Building Materials' 'Airlines' 'Apparel Manufacturing'
 'Oil & Gas R

In [6]:

### Next I perform calculations for MF_ROC and MF_EY ###
represents_earning = 'Operating Income'
# represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,AAA.VN,2024-08-19,Packaging & Containers,Consumer Cyclical,4.175762e+12,9172.666,1.221911e+13,3.567132e+12,7.232351e+11,4.287936e+11,2024-06-30,0.049560,0.102686
1,AGG.VN,2024-08-19,Real Estate - Diversified,Real Estate,4.581222e+12,1458.605,8.197170e+12,4.432107e+12,9.637858e+11,6.707227e+11,2024-06-30,0.178144,0.146407
2,AGR.VN,2024-08-19,Capital Markets,Financial Services,4.603545e+12,947.807,3.626322e+12,1.230129e+12,1.880748e+11,1.614586e+11,2024-06-30,0.067381,0.035073
3,ANV.VN,2024-08-19,Farm Products,Consumer Defensive,5.936467e+12,743.810,4.912103e+12,1.895838e+12,1.563800e+11,1.703607e+11,2024-06-30,0.056481,0.028697
4,ASM.VN,2024-08-19,Real Estate Services,Real Estate,1.535696e+13,5299.314,2.064296e+13,8.243803e+12,1.000054e+12,8.072538e+11,2024-06-30,0.065106,0.052566
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,VNS.VN,2024-08-19,Railroads,Industrials,6.332544e+11,5809.262,1.599091e+12,1.770284e+11,1.213613e+11,5.040991e+10,2024-06-30,0.035448,0.079605
129,VPI.VN,2024-08-19,Real Estate Services,Real Estate,2.255011e+13,2515.546,1.179053e+13,2.958304e+12,4.830607e+11,1.919088e+11,2024-06-30,0.021728,0.008510
130,VRE.VN,2024-08-19,Real Estate Services,Real Estate,4.455697e+13,441.498,5.232844e+13,5.501841e+12,5.974887e+12,4.455608e+12,2024-06-30,0.095151,0.099998
131,VSC.VN,2024-08-19,Marine Shipping,Industrials,4.653372e+12,8194.963,6.488632e+12,3.501181e+11,5.594576e+11,5.385555e+11,2024-06-30,0.087734,0.115734


In [7]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [8]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [9]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
7,CAV.VN,2024-08-19,Electrical Equipment & Parts,Industrials,5.813646e+12,5356.419,4.449233e+12,2.721354e+12,5.754639e+11,7.534632e+11,2024-03-31,0.436063,0.129603,68.0,67.0,135.0


In [10]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,CAV.VN,2024-08-19,Electrical Equipment & Parts,Industrials,5.813646e+12,5356.419,4.449233e+12,2.721354e+12,5.754639e+11,7.534632e+11,2024-03-31,0.436063,0.129603,68.0,67.0,135.0
1,TLG.VN,2024-08-19,Business Equipment & Supplies,Industrials,3.553712e+12,12073.567,3.272037e+12,8.561575e+11,5.459535e+11,4.997423e+11,2024-06-30,0.206857,0.140625,60.0,69.0,129.0
2,NSC.VN,2024-08-19,Farm Products,Consumer Defensive,1.703220e+12,19458.220,2.417369e+12,9.686115e+11,2.924184e+11,2.959285e+11,2024-06-30,0.204264,0.173746,58.0,70.0,128.0
3,RAL.VN,2024-08-19,Electrical Equipment & Parts,Industrials,5.938980e+12,48445.080,8.429511e+12,5.327405e+12,7.500414e+11,7.363417e+11,2024-06-30,0.237368,0.123985,62.0,65.0,127.0
4,PDN.VN,2024-08-19,Marine Shipping,Industrials,3.726592e+12,16691.008,1.530779e+12,3.924674e+11,4.209456e+11,4.048663e+11,2024-06-30,0.355673,0.108642,67.0,59.0,126.0
5,TDP.VN,2024-08-19,Packaging & Containers,Consumer Cyclical,2.286270e+12,680.470,3.688050e+12,2.470866e+12,3.017643e+11,2.696474e+11,2024-06-30,0.221534,0.117942,61.0,61.0,122.0
6,TRA.VN,2024-08-19,Drug Manufacturers—Specialty & Generic,Healthcare,2.863447e+12,13961.774,1.993973e+12,4.721818e+11,3.344724e+11,3.057976e+11,2024-06-30,0.200946,0.106794,56.0,58.0,114.0
7,SCS.VN,2024-08-19,Airports & Air Services,Industrials,7.239870e+12,9457.815,1.534733e+12,1.745608e+11,5.321912e+11,6.014480e+11,2024-06-30,0.442185,0.083074,69.0,45.0,114.0
8,DVP.VN,2024-08-19,Marine Shipping,Industrials,1.843970e+12,29700.762,1.646404e+12,3.477916e+11,5.441049e+11,2.192655e+11,2024-06-30,0.168846,0.118909,51.0,62.0,113.0
9,PVT.VN,2024-08-19,Marine Shipping,Industrials,1.301282e+13,13735.590,1.756653e+13,3.404632e+12,1.886064e+12,1.809578e+12,2024-06-30,0.127778,0.139061,43.0,68.0,111.0


In [11]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,PAN.VN,2024-08-19,Farm Products,Consumer Defensive,9.098448e+12,58372.652,2.336437e+13,1.421845e+13,1.528646e+12,1.111872e+12,2024-06-30,0.121570,0.122205,41.0,64.0,105.0
1,RAL.VN,2024-08-19,Electrical Equipment & Parts,Industrials,5.938980e+12,48445.080,8.429511e+12,5.327405e+12,7.500414e+11,7.363417e+11,2024-06-30,0.237368,0.123985,62.0,65.0,127.0
2,VCF.VN,2024-08-19,Packaged Foods,Consumer Defensive,4.793270e+12,46951.130,2.828088e+12,5.242615e+11,5.604234e+11,4.556279e+11,2024-06-30,0.197770,0.095056,55.0,55.0,110.0
3,DVP.VN,2024-08-19,Marine Shipping,Industrials,1.843970e+12,29700.762,1.646404e+12,3.477916e+11,5.441049e+11,2.192655e+11,2024-06-30,0.168846,0.118909,51.0,62.0,113.0
4,DGC.VN,2024-08-19,Specialty Chemicals,Basic Materials,3.138369e+13,26960.910,1.534951e+13,1.956650e+12,3.414106e+12,2.744504e+12,2024-06-30,0.204923,0.087450,59.0,49.0,108.0
5,DPM.VN,2024-08-19,Chemicals,Basic Materials,6.246662e+12,24798.598,1.573967e+13,3.402337e+12,7.783246e+11,5.117890e+11,2024-06-30,0.041483,0.081930,13.0,42.0,55.0
6,BBC.VN,2024-08-19,Confectioners,Consumer Defensive,8.098462e+11,24619.533,1.980240e+12,5.372748e+11,1.493731e+11,1.012367e+11,2024-06-30,0.070159,0.125007,19.0,66.0,85.0
7,DHG.VN,2024-08-19,Drug Manufacturers—Specialty & Generic,Healthcare,1.189299e+13,20739.660,6.155169e+12,1.832151e+12,9.692210e+11,8.817054e+11,2024-06-30,0.203956,0.074137,57.0,38.0,95.0
8,DCM.VN,2024-08-19,Agricultural Inputs,Basic Materials,9.841514e+12,20077.055,1.683075e+13,6.074443e+12,1.648694e+12,9.118717e+11,2024-06-30,0.084776,0.092656,25.0,54.0,79.0
9,NSC.VN,2024-08-19,Farm Products,Consumer Defensive,1.703220e+12,19458.220,2.417369e+12,9.686115e+11,2.924184e+11,2.959285e+11,2024-06-30,0.204264,0.173746,58.0,70.0,128.0
